<a href="https://colab.research.google.com/github/aderdouri/PINNs/blob/master/Tutorials/Inverse_pinns_Navier_Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Density and Contour Plots

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the PINN model
class PINN(nn.Module):
    def __init__(self, input_dim=3, output_dim=3, hidden_layers=8, hidden_units=64):
        super(PINN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_units))
        layers.append(nn.Tanh())  # Activation function

        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(hidden_units, hidden_units))
            layers.append(nn.Tanh())

        layers.append(nn.Linear(hidden_units, output_dim))  # Output: [u, v, p]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Define the Navier-Stokes loss
def navier_stokes_loss(model, x, y, t, nu, C1, C2):
    inputs = torch.cat([x, y, t], dim=1).requires_grad_(True)
    outputs = model(inputs)

    u, v, p = outputs[:, 0:1], outputs[:, 1:2], outputs[:, 2:3]

    # First-order derivatives
    u_t = torch.autograd.grad(u, inputs, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0][:, 2:3]
    u_x = torch.autograd.grad(u, inputs, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0][:, 0:1]
    u_y = torch.autograd.grad(u, inputs, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0][:, 1:2]
    v_t = torch.autograd.grad(v, inputs, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0][:, 2:3]
    v_x = torch.autograd.grad(v, inputs, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0][:, 0:1]
    v_y = torch.autograd.grad(v, inputs, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0][:, 1:2]
    p_x = torch.autograd.grad(p, inputs, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0][:, 0:1]
    p_y = torch.autograd.grad(p, inputs, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0][:, 1:2]

    # Second-order derivatives
    u_xx = torch.autograd.grad(u_x, inputs, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0][:, 0:1]
    u_yy = torch.autograd.grad(u_y, inputs, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0][:, 1:1]
    v_xx = torch.autograd.grad(v_x, inputs, grad_outputs=torch.ones_like(v_x), retain_graph=True, create_graph=True)[0][:, 0:1]
    v_yy = torch.autograd.grad(v_y, inputs, grad_outputs=torch.ones_like(v_y), retain_graph=True, create_graph=True)[0][:, 1:1]

    # Continuity equation
    continuity = u_x + v_y

    # X-momentum equation
    x_momentum = (
        u_t
        + C1 * (u * u_x + v * u_y)
        + p_x
        - C2 * (u_xx + u_yy)
    )

    # Y-momentum equation
    y_momentum = (
        v_t
        + C1 * (u * v_x + v * v_y)
        + p_y
        - C2 * (v_xx + v_yy)
    )

    # Physics loss
    physics_loss = (
        torch.mean(continuity**2) +
        torch.mean(x_momentum**2) +
        torch.mean(y_momentum**2)
    )

    return physics_loss

# Training function
def train_pinn(model, x_train, y_train, t_train, u_train, v_train, p_train, epochs=5000, lr=1e-3, nu=0.01, C1=1.0, C2=1.0):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    mse_loss = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()

        # Data loss
        inputs = torch.cat([x_train, y_train, t_train], dim=1)
        outputs = model(inputs)
        u_pred, v_pred, p_pred = outputs[:, 0:1], outputs[:, 1:2], outputs[:, 2:3]
        data_loss = (
            mse_loss(u_pred, u_train) +
            mse_loss(v_pred, v_train) +
            mse_loss(p_pred, p_train)
        )

        # Physics loss
        physics_loss = navier_stokes_loss(model, x_train, y_train, t_train, nu, C1, C2)

        # Total loss
        loss = data_loss + physics_loss
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}, Data Loss: {data_loss.item()}, Physics Loss: {physics_loss.item()}")

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PINN().to(device)

# Assuming x_train, y_train, t_train, u_train, v_train, p_train are preloaded tensors
x_train = x_train.to(device)
y_train = y_train.to(device)
t_train = t_train.to(device)
u_train = u_train.to(device)
v_train = v_train.to(device)
p_train = p_train.to(device)

train_pinn(model, x_train, y_train, t_train, u_train, v_train, p_train, epochs=5000, lr=1e-3)
